In [1]:
#Chargement des librairies
import pandas as pd

In [2]:
#Configuration
srcFile = "../data/processed/tempo_202510201126/cerisiers_processed.parquet"

In [4]:
#Chargement du dataset
df = pd.read_parquet(srcFile, engine="fastparquet")

In [5]:
df

,latitude,longitude,n_avant_floraison,jour_n,temps_thermique5,temps_thermique10,date
0,43.857086,4.63403,103,1,5.70,0.7,20160101.0
1,43.857086,4.63403,102,2,12.20,2.2,20160102.0
2,43.857086,4.63403,101,3,13.70,2.2,20160103.0
3,43.857086,4.63403,100,4,17.30,2.2,20160104.0
4,43.857086,4.63403,99,5,21.20,2.2,20160105.0
...,...,...,...,...,...,...,...
553131,44.188300,5.17206,4,105,347.95,55.4,20210415.0
553132,44.188300,5.17206,3,106,350.05,55.4,20210416.0
553133,44.188300,5.17206,2,107,353.45,55.4,20210417.0
553134,44.188300,5.17206,1,108,356.25,55.4,20210418.0


In [7]:
df.describe()

,latitude,longitude,n_avant_floraison,jour_n,temps_thermique5,temps_thermique10,date
count,553136.000000,553136.000000,553136.000000,553136.000000,553136.000000,553136.000000,5.531360e+05
mean,45.497736,1.978872,50.104578,51.104578,146.576265,26.319562,2.008924e+07
std,1.923495,2.636523,32.426899,32.426899,206.307218,107.324201,1.379395e+05
min,42.476616,-4.392580,0.000000,1.000000,0.000000,0.000000,1.950010e+07
25%,44.186000,-0.576226,24.000000,25.000000,41.500000,1.450000,2.001033e+07
50%,44.785700,1.827440,49.000000,50.000000,104.950000,9.500000,2.015011e+07
75%,46.980000,4.557690,73.000000,74.000000,201.800000,26.400000,2.018031e+07
max,50.731144,7.752111,348.000000,349.000000,4162.600000,2560.600000,2.025043e+07


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 553136 entries, 0 to 553135
Data columns (total 7 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   latitude           553136 non-null  float64
 1   longitude          553136 non-null  float64
 2   n_avant_floraison  553136 non-null  int64  
 3   jour_n             553136 non-null  int64  
 4   temps_thermique5   553136 non-null  float64
 5   temps_thermique10  553136 non-null  float64
 6   date               553136 non-null  float64
dtypes: float64(5), int64(2)
memory usage: 29.5 MB


In [9]:
#Séparation X et y
y = df["n_avant_floraison"]
X = df.drop(["n_avant_floraison", "date"], axis=1)

In [10]:
y

0         103
1         102
2         101
3         100
4          99
         ... 
553131      4
553132      3
553133      2
553134      1
553135      0
Name: n_avant_floraison, Length: 553136, dtype: int64

In [11]:
X

,latitude,longitude,jour_n,temps_thermique5,temps_thermique10
0,43.857086,4.63403,1,5.70,0.7
1,43.857086,4.63403,2,12.20,2.2
2,43.857086,4.63403,3,13.70,2.2
3,43.857086,4.63403,4,17.30,2.2
4,43.857086,4.63403,5,21.20,2.2
...,...,...,...,...,...
553131,44.188300,5.17206,105,347.95,55.4
553132,44.188300,5.17206,106,350.05,55.4
553133,44.188300,5.17206,107,353.45,55.4
553134,44.188300,5.17206,108,356.25,55.4


In [12]:
#Séparation du jeu d'entrâinement et de test

from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = train_test_split(
	X,
	y, 
	train_size=0.8, 
	test_size=0.2,
	random_state=42
)

In [14]:
print(f"Nombre de ligne dans le jeu d'entraînement : {X_train.shape[0]}")
print(f"Nombre de ligne dans le jeu de test : {X_valid.shape[0]}")

Nombre de ligne dans le jeu d'entraînement : 442508
Nombre de ligne dans le jeu de test : 110628


In [15]:
from sklearn.ensemble import RandomForestRegressor
model = RandomForestRegressor(n_estimators=100, random_state=0)

In [17]:
from sklearn.metrics import mean_absolute_error
from sklearn.pipeline import Pipeline

# Bundle preprocessing and modeling code in a pipeline
my_pipeline = Pipeline(
    steps=[
            ('model', model)
    ]
)

In [18]:
# Preprocessing of training data, fit model
my_pipeline.fit(X_train, y_train)

,steps,"[('model', ...)]"
,transform_input,None
,memory,None
,verbose,False
,n_estimators,100
,criterion,'squared_error'
,max_depth,None
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,1.0


In [52]:
#Sauvegarder le modèle
import joblib

joblib.dump(my_pipeline, 'modele2.pkl')

['modele2.pkl']

In [51]:
# Preprocessing of validation data, get predictions
y_pred_valid = my_pipeline.predict(X_valid)
y_pred_train = my_pipeline.predict(X_train)

In [30]:
# Evaluate the model
from sklearn.metrics import mean_squared_error, r2_score

score_mae_valid = mean_absolute_error(y_valid, y_pred_valid)
score_mae_train = mean_absolute_error(y_train, y_pred_train)
print('MAE (jeu de validation) :', score_mae_valid)
print('MAE (jeu d\'enytraînement) :', score_mae_train)

# Erreur quadratique moyenne (MSE)
score_mse_train = mean_squared_error(y_train, y_pred_train)
score_mse_valid = mean_squared_error(y_valid, y_pred_valid)
print('MSE (jeu de validation) :', score_mse_valid)
print('MSE (jeu d\'enytraînement) :', score_mse_train)

# Coefficient de détermination (R²) : de 0 à 1, plus c'est proche de 1, mieux c'est
r2_train = r2_score(y_train, y_pred_train)
r2_valid = r2_score(y_valid, y_pred_valid)
print('R2 (jeu de validation) :', r2_valid)
print('R2 (jeu d\'enytraînement) :', r2_train)





MAE (jeu de validation) : 4.403533424074006
MAE (jeu d'enytraînement) : 3.9340949103883283
MSE (jeu de validation) : 56.98222330266349
MSE (jeu d'enytraînement) : 46.22239991885678
R2 (jeu de validation) : 0.9455971069975255
R2 (jeu d'enytraînement) : 0.9560839459156244


In [47]:
#Exemples de prédictions

for index in range(0, 20):
    print(f"Valeur réelle : {y_valid.iloc[index]} - valeur prédite : {y_pred_valid[index]}")

Valeur réelle : 51 - valeur prédite : 48.43316551256241
Valeur réelle : 82 - valeur prédite : 84.49970125626011
Valeur réelle : 48 - valeur prédite : 41.30055158730159
Valeur réelle : 34 - valeur prédite : 33.124793650793656
Valeur réelle : 15 - valeur prédite : 15.0
Valeur réelle : 23 - valeur prédite : 16.415
Valeur réelle : 77 - valeur prédite : 88.47055420001152
Valeur réelle : 27 - valeur prédite : 32.99348157799988
Valeur réelle : 69 - valeur prédite : 62.61502544507134
Valeur réelle : 40 - valeur prédite : 30.084806527419104
Valeur réelle : 23 - valeur prédite : 22.3897162004662
Valeur réelle : 44 - valeur prédite : 39.36350000000001
Valeur réelle : 53 - valeur prédite : 55.67241252573185
Valeur réelle : 70 - valeur prédite : 66.05733439771804
Valeur réelle : 59 - valeur prédite : 67.71132270551858
Valeur réelle : 111 - valeur prédite : 108.40412698412698
Valeur réelle : 60 - valeur prédite : 59.63
Valeur réelle : 0 - valeur prédite : 0.13
Valeur réelle : 94 - valeur prédite : 9